# Setting Up GPU and ML Management Tools

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# If you don't want your script to sync to the cloud
os.environ['WANDB_IGNORE_GLOBS'] = "*.patch"

In [2]:
from comet_ml import Experiment
import wandb

# Importing Modules

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
from joblib import dump, load
from  IPython import display
import shutil
import tempfile
import pathlib
import sys

# This allows us to import the nucml utilities
sys.path.append("..")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
import nucml.exfor.data_utilities as exfor_utils
import nucml.endf.data_utilities as endf_utils
import nucml.plot.plotting_utilities as plot_utils
import nucml.datasets as nuc_data
import nucml.ace.data_utilities as ace_utils
import nucml.model.model_building_nn as model_tools
importlib.reload(model_tools)
importlib.reload(exfor_utils)
importlib.reload(endf_utils)
importlib.reload(plot_utils)
importlib.reload(nuc_data)
importlib.reload(ace_utils)
print("Finish importing scripts.")

Finish importing scripts.


In [5]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [6]:
%tensorboard --logdir wandb --port=6006

# Loading EXFOR data (Energy in eV and Data in b)

In [6]:
# df, x_train, x_test, y_train, y_test, to_scale, scaler = nuc_data.load_exfor(log=True, basic=1, num=True, low_en=True, scaler_dir='../ML_Data/Models/NN/scaler_ext.pkl')
df, x_train, x_test, y_train, y_test, to_scale, scaler = nuc_data.load_exfor(
    log=True, basic=1, num=True, low_en=True, scaling_type="std")

C:\Users\Pedro\Desktop\ML_Nuclear_Data\ML_Data\EXFOR_neutrons\EXFOR_neutrons_MF3_AME_no_NaNRaw.csv
Reading data into dataframe...
 MODE: neutrons
 LOW ENERGY: True
 LOG: True
 BASIC: 1
 SCALER: std
Data read into dataframe with shape:  (4184245, 19)
Dropping unnecessary features and one-hot encoding categorical columns...
Splitting dataset into training and testing...
Normalizing dataset...
Fitting new scaler.
Finished. Resulting dataset has shape  (4184245, 58) 
Training and Testing dataset shapes are (3765820, 57) and (418425, 57) respesctively.


In [ ]:
nn_saving_dir = '../ML_Data/Models/NN/'

dump(scaler, open(os.path.join(nn_saving_dir, 'scaler_ext_std.pkl'), 'wb'))

# Sampling Chlorine (n,p) and U-233(n,f) Data for Visualization

The model will be trained in the entire data therefore getting the total MSE. As an example, we will see the model predictions for both the mentioned reactions. 

In [7]:
kwargs = {"nat_iso": "I", "one_hot": True, "scale": True, "scaler": scaler, "to_scale": to_scale}

chlorine_35_np = exfor_utils.load_exfor_samples(df, 17, 35, "MT_103", **kwargs)
uranium_235_nt = exfor_utils.load_exfor_samples(df, 92, 235, "MT_1", **kwargs)
uranium_233_nf = exfor_utils.load_exfor_samples(df, 92, 233, "MT_18", **kwargs)
uranium_233_nt = exfor_utils.load_exfor_samples(df, 92, 233, "MT_1", one_hot=True)

Extracting samples from dataframe.
Scaling dataset...
EXFOR extracted DataFrame has shape:  (215, 58)
Extracting samples from dataframe.
Scaling dataset...
EXFOR extracted DataFrame has shape:  (40108, 58)
Extracting samples from dataframe.
Scaling dataset...
EXFOR extracted DataFrame has shape:  (94567, 58)
Extracting samples from dataframe.
EXFOR extracted DataFrame has shape:  (33384, 58)


In [8]:
kwargs = {"nat_iso": "I", "one_hot": True, "scale": True, "scaler": scaler, "to_scale": to_scale}
uranium = exfor_utils.load_exfor_element(df, 92, **kwargs)

Extracting samples from dataframe.
Scaling dataset...
EXFOR extracted DataFrame has shape:  (468123, 58)


# Newly Measured Chlorine (n,p) Unseen Data 

These data points are not in the current EXFOR data package and will be used to test the new modeling capabilities.

In [9]:
new_cl_data_kwargs = {"Z":17, "A":35, "MT":"MT_103", "log":True, "scale":False, "scaler":scaler, "to_scale":to_scale}
new_cl_data = exfor_utils.load_exfor_newdata("../EXFOR/New_Data/Chlorine_Data/new_cl_np.csv", df, **new_cl_data_kwargs)
new_cl_data.head()

Extracting samples from dataframe.
EXFOR extracted DataFrame has shape:  (215, 58)
Expanded Dataset has shape:  (12, 58)


,Energy,Data,Target_Protons,Target_Neutrons,Target_Mass_Number,Target_Radius,Target_Neut_Rad_Ratio,Target_Mass_Excess,Target_Binding_Energy,Target_B_Decay_Energy,Target_Atomic_Mass_Micro,Target_S(2n),Target_S(n),Target_S(p),MT_1,MT_102,MT_16,MT_17,MT_2,MT_3,MT_4,MT_101,MT_103,MT_104,MT_41,MT_9000,MT_105,MT_32,MT_51,MT_33,MT_107,MT_24,MT_155,MT_158,MT_159,MT_108,MT_29,MT_1108,MT_113,MT_106,MT_22,MT_1003,MT_9001,MT_28,MT_111,MT_203,MT_2103,MT_112,MT_37,MT_161,MT_152,MT_153,MT_18,MT_160,Frame_L,Frame_C,Target_Flag_I,Target_Flag_N
0,6.383815,-1.779892,17,18,35,4.088833,0.195655,-29013.528,8520.278,-5966.243,3.496885e+07,24152.83,12644.76,6370.81,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,6.383815,-1.707744,17,18,35,4.088833,0.195655,-29013.528,8520.278,-5966.243,3.496885e+07,24152.83,12644.76,6370.81,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
2,6.401401,-1.583359,17,18,35,4.088833,0.195655,-29013.528,8520.278,-5966.243,3.496885e+07,24152.83,12644.76,6370.81,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,6.401401,-1.590067,17,18,35,4.088833,0.195655,-29013.528,8520.278,-5966.243,3.496885e+07,24152.83,12644.76,6370.81,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,6.411620,-1.350665,17,18,35,4.088833,0.195655,-29013.528,8520.278,-5966.243,3.496885e+07,24152.83,12644.76,6370.81,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


# ENDF Evaluation Cross Section Data for Chlorine (n,p) Reaction

These data points will serve to plot the current ENDFb5 data and compare it to the newly measured points along with the predictions

In [10]:
# ENDF EXTRACTION IMPLEMENTED INTO ANOTHER FUNCTION, NO NEED TO OBTAIN MANUALLY IN FUTURE VERSIONS

endf_cl = nuc_data.load_endf("Cl035", "MT103", log=True)
endf_u  = nuc_data.load_endf("U233", "MT018", log=True)
endf_cl_37 = nuc_data.load_endf("Cl037", "MT102", log=True)
endf_fe_56 = nuc_data.load_endf("Fe056", "MT002", log=True)

C:\Users\Pedro\Desktop\ML_Nuclear_Data\ML_Data\ENDF_neutrons\Cl035\endfb8.0\tables\xs\n-Cl035-MT103.endfb8.0
Converting MeV to eV...
Convering mb to b...
Finish reading ENDF data with shape:  (8791, 2)
C:\Users\Pedro\Desktop\ML_Nuclear_Data\ML_Data\ENDF_neutrons\U233\endfb8.0\tables\xs\n-U233-MT018.endfb8.0
Converting MeV to eV...
Convering mb to b...
Finish reading ENDF data with shape:  (15345, 2)
C:\Users\Pedro\Desktop\ML_Nuclear_Data\ML_Data\ENDF_neutrons\Cl037\endfb8.0\tables\xs\n-Cl037-MT102.endfb8.0
Converting MeV to eV...
Convering mb to b...
Finish reading ENDF data with shape:  (20121, 2)
C:\Users\Pedro\Desktop\ML_Nuclear_Data\ML_Data\ENDF_neutrons\Fe056\endfb8.0\tables\xs\n-Fe056-MT002.endfb8.0
Converting MeV to eV...
Convering mb to b...
Finish reading ENDF data with shape:  (46021, 2)


# Reading ACE Energies

In [11]:
# ACE EXTRACTION IMPLEMENTED INTO ANOTHER FUNCTION, NO NEED TO OBTAIN MANUALLY IN FUTURE VERSIONS

ace_u = ace_utils.get_energies("92233", ev=True, log=True)
ace_cl = ace_utils.get_energies("17035", ev=True, log=True)
ace_cl_37 = ace_utils.get_energies("17037", ev=True, log=True)
ace_fe_56 = ace_utils.get_energies("26056", ev=True, log=True)

# Setting up Chlorine and Uranium Prediction Tools

In [12]:
order = {
    "3":"endf", 
    "1":"exfor_ml_original", 
    "2":"exfor_ml", 
    "4":"exfor_new"}

def run_chlorine(model):
    cl_kwargs =  {"Z":17, "A":35, "MT":"MT_103", "clf_type":"tf", "scaler":scaler, "to_scale":to_scale, "html":False,
                  "e_array":"ace", "log":True, "show":False, "render":False, "save":False, "inv_trans":True}
    results_cl = exfor_utils.predicting_nuclear_xs_v2(df, clf=model, new_data=new_cl_data, get_endf=True, **cl_kwargs)
    fig = plot_utils.plotly_ml_results(results_cl, show=True)
    return fig

def run_uranium(model):
    u_kwargs =  {"Z":92, "A":233, "MT":"MT_18", "clf_type":"tf", "scaler":scaler, "to_scale":to_scale, "html":False,
                  "e_array":"ace", "log":True, "show":False, "render":False, "save":False, "inv_trans":True}
    results_u = exfor_utils.predicting_nuclear_xs_v2(df, clf=model, get_endf=True, **u_kwargs)
    fig = plot_utils.plotly_ml_results(results_u, order_dict=order, show=True)
    return fig

# Using TensorFlow

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
print(tf.__version__)

2.1.0


In [14]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


### Setting up Output Folder for Checkpoints and Checkpoint Callback

In [15]:
histories = {}

In [16]:
# x_train_t = x_train[:10000]
# y_train_t = y_train[:10000]
# x_test_t = x_test[:10000]
# y_test_t = y_test[:10000]

In [17]:
COMMET_PROJECT_NAME = "ml-nuclear-data"
WANDB_PROJECT_NAME = "ml-nuclear-data"
GROUP_NAME = "EXFOR_Basic_1" # CHANGED
NUM_FEATURES = x_train.shape[1]
BATCH_SIZE = 300
EPOCHS = 100
ACTIVATION = "relu"
OPTIMIZER = "ITD"
LR_DECAY_EPOCHS = 10
STEPS_PER_EPOCH = len(x_train) // BATCH_SIZE
SCALER_TYPE = "std"

In [18]:
for num_layers_to_try in [5]: # np.linspace(1,5, 5, dtype=int): # FOR 1 TO 5 LAYERS   
    for num_units in [100]: # [55, 70, 85]:  # FOR 100 TO 500 NEURONS
        params = {}
        
        # ------------------------------------------ TENSORFLOW ----------------------------------------------
        # BUILDING MODEL AND LOGGING NUMBER OF LAYERS AND UNITS
        model = tf.keras.Sequential() 
        for num_layers in np.linspace(1, num_layers_to_try, num_layers_to_try, dtype=int):
            model.add(tf.keras.layers.Dense(num_units, activation='relu'))
            params["Layer_{}_Units".format(num_layers)] = num_units
        model.add(tf.keras.layers.Dense(1))
        model.build((None, NUM_FEATURES))
        # ----------------------------------------------------------------------------------------------------
        
        # --------------------------------------- COMET and WANDB --------------------------------------------
        # DEFINING NAME OF RUN/EXPERIMENT
        RUN_NAME = '{}_Layers_{}_Units_B1_{}'.format(num_layers_to_try, num_units, SCALER_TYPE) 
        
        # CREATING AND ADDING HYPERPARAMETRES TO PARAMS DICT FOR LOGGING
        TO_ADD = {'Batch_Size':BATCH_SIZE,
                  'Activation':ACTIVATION,
                  'Optimizer':OPTIMIZER,
                  'LR_Decay_Epochs':LR_DECAY_EPOCHS,
                  'Steps_per_Epoch':STEPS_PER_EPOCH,
                  "Layer_{}_Units".format(num_layers + 1):1,
                  "Number_of_Layers":num_layers + 1}
        params.update(TO_ADD)
        # ----------------------------------------------------------------------------------------------------
        
        # -------------------------------------------- WANDB -------------------------------------------------
        # INITIATING WANDB RUN AND UPDATING CONFIG WITH HYPERPARAMETER VALUES
        wandb.init(project=WANDB_PROJECT_NAME, name=RUN_NAME, sync_tensorboard=True, reinit=True, 
                   group=GROUP_NAME, id=RUN_NAME) # config=TO_ADD)
        os.environ['WANDB_NOTEBOOK_NAME'] = '5_NN_EXFOR_v2_neutrons'
        wandb.config.update(TO_ADD)
        # ----------------------------------------------------------------------------------------------------
        
        # -------------------------------------------- COMET -------------------------------------------------
        # COMET ML WORKS BY INITIATING EXPERIMENT AND BY LOGGING A DICTIONARY
        comet_experiment = Experiment(api_key="fJ314gp1hdwdnn3632Z3VCZ2B",
                        project_name=COMMET_PROJECT_NAME, workspace="pedrojrv")
        comet_experiment.set_name(RUN_NAME)
        comet_experiment.log_parameters(params) # LOGGING HYPERPARAMETERS TO COMET EXPERIMENT
        LOGGING_DIR_NAME =  wandb.run.dir  # SETTING DIRECTORY TO SAVE CHECKPOINTS, TENSORBOARD, AND CSV
        # ----------------------------------------------------------------------------------------------------
        
        # ------------------------------------------ TENSORFLOW ----------------------------------------------
        histories[RUN_NAME] = model_tools.compile_and_fit(model, RUN_NAME, 
            x_train.values, y_train.values, x_test.values, y_test.values, 
            BATCH_SIZE=BATCH_SIZE, max_epochs=EPOCHS, DECAY_EPOCHS=LR_DECAY_EPOCHS, lr_method=OPTIMIZER, 
            logs_dir_name=LOGGING_DIR_NAME, append_wandb=True, comet=True, comet_exp=comet_experiment)
        # ----------------------------------------------------------------------------------------------------
        
        # ------------------------------------------ WANDB ----------------------------------------------
        # LOGGING CHLORINE PLOTLY PLOT
        fig_to_log_cl = run_chlorine(histories[RUN_NAME].model)
        fig_to_log_u = run_uranium(histories[RUN_NAME].model)
        wandb.log({'Chlorine_35_NP': fig_to_log_cl})
        wandb.log({'Uranium_233_NF': fig_to_log_u})
        
        # EVERY RUN WANDB CREATES A DIFF.PATCH, WE REMOVE IT TO SAVE TIME WHEN UPLAODING ASSET WITH COMET
        os.remove(os.path.join(LOGGING_DIR_NAME, "diff.patch"))
        # ----------------------------------------------------------------------------------------------------
        
        # -------------------------------------------- COMET -------------------------------------------------
        # COMET DOES NOT SAVE MODEL OR ANY OTHER DATA FILES, MUST BE LOGGED MANUALLY
        # HERE WE LOG THE MODEL, THE TRAINING CSV, THE ENTIRE WANDB FOLDER,
        # A PIL VERSION OF THE PLOTLY IMAGE AND THE DATASET VERSION
        comet_experiment.log_model(RUN_NAME, os.path.join(wandb.run.dir, "model-best.h5")) 
        comet_experiment.log_asset_folder(LOGGING_DIR_NAME) 
        comet_experiment.log_table(os.path.join(wandb.run.dir, "training_metrics.csv")) 
        comet_experiment.log_dataset_info(name="EXFOR_Basic", version="1")
        
        # LOGGING FIGURES TO COMET (DOES NOT ACCEPT PLOTLY)
        pil_fig_cl = plot_utils.plotly_fig2pil(fig_to_log_cl)
        pil_fig_u = plot_utils.plotly_fig2pil(fig_to_log_u)
        comet_experiment.log_image(pil_fig_cl, name="Chlorine_35_NP") # MAPTLOTLIB PYPLOT USING LOG_FIGURE
        comet_experiment.log_image(pil_fig_u, name="Uranium_233_NF")
        # ----------------------------------------------------------------------------------------------------
        
        wandb.join()
        comet_experiment.end()

COMET INFO: old comet version (3.1.12) detected. current: 3.1.13 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET ERROR: Failed to log git patch
COMET ERROR: Failed to log git patch
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/pedrojrv/ml-nuclear-data/b556a0b45b394bbdb7f8b25cb1135a4d

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE
COMET INFO: Ignoring automatic log_parameter('do_validation') because 'keras:do_validation' is in COMET_LOGGING_PARAMETERS_IGNORE
COMET INFO: Ignoring automatic log_metric('batch_batch') because 'keras:batch_batch' is in COMET_LOGGING_METRICS_IGNORE
COMET INFO: Ignoring automatic log_metric('batch_size') because 'keras:batch_size' is in COMET_LOGGING_METRICS_IGNORE



Epoch: 0, loss:0.1540,  mae:0.2459,  mse:0.1540,  val_loss:0.1378,  val_mae:0.2317,  val_mse:0.1378,  
....

KeyError: 'val_loss'

In [19]:
wandb.join()
comet_experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/pedrojrv/ml-nuclear-data/b556a0b45b394bbdb7f8b25cb1135a4d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_batch_loss [3995]   : (0.06486985832452774, 1.3093023300170898)
COMET INFO:     train_batch_mae [3995]    : (0.19620899856090546, 0.9269266724586487)
COMET INFO:     train_batch_mse [3995]    : (0.11206133663654327, 1.3093023300170898)
COMET INFO:     train_epoch_duration [3]  : (51.57799999997951, 54.203999999997905)
COMET INFO:     train_loss [3]            : (0.12431211271066338, 0.15403235541660174)
COMET INFO:     train_mae [3]             : (0.2176709771156311, 0.2459419220685959)
COMET INFO:     train_mse [3]             : (0.12431201338768005, 0.15403242409229279)
COMET INFO:     train_val_loss [3]        : (0.122899341

In [ ]:
# TAG = "ASYM500_5_v3"
# params = {}

# # ------------------------------------------ TENSORFLOW ----------------------------------------------
# # BUILDING MODEL AND LOGGING NUMBER OF LAYERS AND UNITS
# params["Layer_1_Units"] = 500
# params["Layer_2_Units"] = 400
# params["Layer_3_Units"] = 300
# params["Layer_4_Units"] = 200
# params["Layer_5_Units"] = 100
# params["Layer_6_Units"] = 50
# params["Layer_7_Units"] = 25
# params["Layer_8_Units"] = 15
# params["Layer_9_Units"] = 10
# params["Layer_10_Units"] = 5
# params["Layer_11_Units"] = 1


# model = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(params["Layer_1_Units"], activation=ACTIVATION), 
#     tf.keras.layers.Dense(params["Layer_2_Units"], activation=ACTIVATION),
#     tf.keras.layers.Dense(params["Layer_3_Units"], activation=ACTIVATION), 
#     tf.keras.layers.Dense(params["Layer_4_Units"], activation=ACTIVATION),
#     tf.keras.layers.Dense(params["Layer_5_Units"], activation=ACTIVATION),
#     tf.keras.layers.Dense(params["Layer_6_Units"], activation=ACTIVATION),
#     tf.keras.layers.Dense(params["Layer_7_Units"], activation=ACTIVATION), 
#     tf.keras.layers.Dense(params["Layer_8_Units"], activation=ACTIVATION), 
#     tf.keras.layers.Dense(params["Layer_9_Units"], activation=ACTIVATION), 
#     tf.keras.layers.Dense(params["Layer_10_Units"], activation=ACTIVATION), 
#     tf.keras.layers.Dense(params["Layer_11_Units"])])
# # ----------------------------------------------------------------------------------------------------

# # --------------------------------------- COMET and WANDB --------------------------------------------
# # DEFINING NAME OF RUN/EXPERIMENT
# RUN_NAME = '{}_Layers_{}_B1_{}'.format(len(params)-1, TAG, SCALER_TYPE) 

# # CREATING AND ADDING HYPERPARAMETRES TO PARAMS DICT FOR LOGGING
# TO_ADD = {'Batch_Size':BATCH_SIZE,
#           'Activation':ACTIVATION,
#           'Optimizer':OPTIMIZER,
#           'LR_Decay_Epochs':LR_DECAY_EPOCHS,
#           'Steps_per_Epoch':STEPS_PER_EPOCH,
#           "Number_of_Layers":len(params)}
# params.update(TO_ADD)
# # ----------------------------------------------------------------------------------------------------

# # -------------------------------------------- WANDB -------------------------------------------------
# # INITIATING WANDB RUN AND UPDATING CONFIG WITH HYPERPARAMETER VALUES
# wandb.init(project=WANDB_PROJECT_NAME, name=RUN_NAME, sync_tensorboard=True, reinit=True, 
#            group=GROUP_NAME, id=RUN_NAME) # config=TO_ADD)
# os.environ['WANDB_NOTEBOOK_NAME'] = '5_NN_EXFOR_v2_neutrons'
# wandb.config.update(TO_ADD)
# # ----------------------------------------------------------------------------------------------------

# # -------------------------------------------- COMET -------------------------------------------------
# # COMET ML WORKS BY INITIATING EXPERIMENT AND BY LOGGING A DICTIONARY
# comet_experiment = Experiment(api_key="fJ314gp1hdwdnn3632Z3VCZ2B",
#                 project_name=COMMET_PROJECT_NAME, workspace="pedrojrv")
# comet_experiment.set_name(RUN_NAME)
# comet_experiment.log_parameters(params) # LOGGING HYPERPARAMETERS TO COMET EXPERIMENT
# LOGGING_DIR_NAME =  wandb.run.dir  # SETTING DIRECTORY TO SAVE CHECKPOINTS, TENSORBOARD, AND CSV
# # ----------------------------------------------------------------------------------------------------

# # ------------------------------------------ TENSORFLOW ----------------------------------------------
# histories[RUN_NAME] = model_tools.compile_and_fit(model, RUN_NAME, 
#     x_train.values, y_train.values, x_test.values, y_test.values, 
#     BATCH_SIZE=BATCH_SIZE, max_epochs=EPOCHS, DECAY_EPOCHS=LR_DECAY_EPOCHS, lr_method=OPTIMIZER, 
#     logs_dir_name=LOGGING_DIR_NAME, append_wandb=True, comet=True, comet_exp=comet_experiment)
# # ----------------------------------------------------------------------------------------------------

# # ------------------------------------------ WANDB ----------------------------------------------
# # LOGGING CHLORINE PLOTLY PLOT
# fig_to_log_cl = run_chlorine(histories[RUN_NAME].model)
# fig_to_log_u = run_uranium(histories[RUN_NAME].model)
# wandb.log({'Chlorine_35_NP': fig_to_log_cl})
# wandb.log({'Uranium_233_NF': fig_to_log_u})

# # EVERY RUN WANDB CREATES A DIFF.PATCH, WE REMOVE IT TO SAVE TIME WHEN UPLAODING ASSET WITH COMET
# os.remove(os.path.join(LOGGING_DIR_NAME, "diff.patch"))
# # ----------------------------------------------------------------------------------------------------

# # -------------------------------------------- COMET -------------------------------------------------
# # COMET DOES NOT SAVE MODEL OR ANY OTHER DATA FILES, MUST BE LOGGED MANUALLY
# # HERE WE LOG THE MODEL, THE TRAINING CSV, THE ENTIRE WANDB FOLDER,
# # A PIL VERSION OF THE PLOTLY IMAGE AND THE DATASET VERSION
# comet_experiment.log_model(RUN_NAME, os.path.join(wandb.run.dir, "model-best.h5")) 
# comet_experiment.log_asset_folder(LOGGING_DIR_NAME) 
# comet_experiment.log_table(os.path.join(wandb.run.dir, "training_metrics.csv")) 
# comet_experiment.log_dataset_info(name="EXFOR_Basic", version="1")

# # LOGGING FIGURES TO COMET (DOES NOT ACCEPT PLOTLY)
# pil_fig_cl = plot_utils.plotly_fig2pil(fig_to_log_cl)
# pil_fig_u = plot_utils.plotly_fig2pil(fig_to_log_u)
# comet_experiment.log_image(pil_fig_cl, name="Chlorine_35_NP") # MAPTLOTLIB PYPLOT USING LOG_FIGURE
# comet_experiment.log_image(pil_fig_u, name="Uranium_233_NF")
# # ----------------------------------------------------------------------------------------------------

# wandb.join()
# comet_experiment.end()

# Loading Model

In [22]:
# def run_chlorine(model):
#     cl_kwargs =  {"Z":17, "A":35, "MT":"MT_103", "clf_type":"tf", "scaler":scaler, "to_scale":to_scale, "html":False,
#                   "e_array":"ace", "log":True, "show":False, "render":False, "save":False}
#     results_cl = exfor_utils.predicting_nuclear_xs_v2(df, clf=model, new_data=new_cl_data, get_endf=True, **cl_kwargs)
#     fig = plot_utils.plotly_ml_results(results_cl, show=True)
#     return fig

# best_model = tf.keras.models.load_model('./wandb/run-20200705_013609-7_Layers_200_Units_B1/model-best.h5')
# best_model_comet = tf.keras.models.load_model('./wandb/run-20200705_013609-7_Layers_200_Units_B1/checkpoints/best_model.hdf5')

# fig_to_log_cl = run_chlorine(best_model)

In [ ]:
# cl_kwargs =  {"Z":17, "A":35, "MT":"MT_103", "clf_type":"tf", "scaler":scaler, "to_scale":to_scale, "html":False,
#               "e_array":"ace", "log":True, "show":True, "render":True, "save":False}
# results_cl = exfor_utils.predicting_nuclear_xs_v2(df, clf=best_model, new_data=new_cl_data, get_endf=True, **cl_kwargs)

# order = {
#     "3":"endf", 
#     "1":"exfor_ml_original", 
#     "2":"exfor_ml", 
#     "4":"exfor_new"}

# u_kwargs =  {"Z":92, "A":233, "MT":"MT_18", "clf_type":"tf", "scaler":scaler, "to_scale":to_scale, "html":False,
#               "e_array":"ace", "log":True, "show":True, "render":True, "save":False}
# results_u = exfor_utils.predicting_nuclear_xs_v2(df, clf=best_model, get_endf=True, order_dict=order, **u_kwargs)